# Tutorial for using MLflow on Puhti

This tutorial will guide you through using MLflow in the Puhti computing environment, offering a streamlined and centralized approach to tracking machine learning experiments. It’s tailored for machine learning practitioners who seek an efficient way to manage and monitor their experiments.

While prior experience with MLflow isn’t necessary, a basic understanding of supercomputing is recommended. We’ll explore the core components of MLflow and demonstrate their application through practical examples. You can follow along with the provided sample code or incorporate your own code into the tutorial.

### What is MLflow?

**MLflow** in an open-source tool for managing machine learning models throughout their life cycle. It has four key components that can be widely utilized, from experimenting to deploying models:

- **Tracking Server** is the core component used for tracking experiments. Results can be viewed and compared through an informative user interface or API.
	
- **Models** is for packaging the models in a unified format, making it easy to move and share them.

- **Model Registry** provides tools for registering and versioning models. The registry can also be managed through the UI.

- **Projects** is for packaging entire ML project code, enabling easy sharing and reproducibility.

By organizing your work into ***experiments*** and ***runs***, MLflow ensures that you can systematically track progress, compare results, and refine your models effectively.

For more info on components visit [MLflow documentation](https://mlflow.org/docs/latest/introduction/index.html#what-is-mlflow).

In [1]:
# Let's make sure we have the latest version of MLflow (>2.15.1) in use. If not, run pip install below:
!mlflow --version
#!pip install --upgrade mlflow

mlflow, version 2.15.1


In [ ]:
import pandas as pd
import requests 
import os

from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras import layers
from keras.datasets import mnist
from keras.utils import to_categorical

import mlflow

---

## Tracking Server & Models

### About Storing Artifacts and Metadata

In the example code, we use local storage for both metadata and artifacts. Additionally, MLflow supports various options for selecting the backend and artifact storage.

- **Backend Store**: This is where ***metadata*** is stored, including information about each run. By default, MLflow saves this metadata locally in the `mlruns` directory. However, it can be configured to use an external database, such as PostgreSQL or MySQL. You can read more about backend stores here: [MLflow Backend Stores](https://mlflow.org/docs/latest/tracking/backend-stores.html#backend-stores).

- **Artifact Store**: This is where the ***artifacts*** generated during runs—such as model weights, model files, and data files—are stored. The **Models component** is used to package these model files in a standardized format. Similar to the backend store, MLflow defaults to using the local `mlruns` directory for artifact storage, but it can be set to use external storage, such as S3 object storage like Allas. For more information, refer to [MLflow Artifact Stores](https://mlflow.org/docs/latest/tracking/artifacts-stores.html) and the [CSC documentation on Allas](https://docs.csc.fi/computing/allas/) and [using Allas with Python and Boto3](https://docs.csc.fi/data/Allas/using_allas/python_boto3/).

In the example code below, local directory is used for the backend and artifact storage.

In [7]:
""""
First, we activate the Tracking Server component:

Start by setting the tracking URI, which defines the path where MLflow will create the 'mlruns' directory to store metadata and artifacts generated during runs. 
If no path is provided, MLflow will create the 'mlruns' directory in the location where the code is executed.

Next, set up an experiment under which the upcoming training runs will be logged. If the experiment does not already exist, it will be created.
"""

# Set tracking URI
project_id = "dac" 
#project_id = "project_2001234" # Insert your project_id here
mlruns_uri = f"/scratch/{project_id}/esternad/mlruns" 
#mlruns_uri = f"scratch/{project_id]/path/for/mlruns" # URI for desired storage
mlflow.set_tracking_uri(mlruns_uri) # https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow-tracing-apis

# Set experiment
experiment_name = "MLflow tutorial"
experiment = mlflow.set_experiment(experiment_name) # https://mlflow.org/docs/latest/python_api/mlflow.html#mlflow-tracing-apis
print(f"Artifacts are saved stored here: {experiment.artifact_location}")

Artifacts are saved stored here: /scratch/dac/esternad/mlruns/645056726564230381


In [10]:
"""
Next, we will run training sessions using MLflow. We'll utilize the autolog function, which automatically logs all the data generated during the run. 
By default, the model will be logged as an artifact, making it easy to access later on. This and other features can be modified in arguments.
In the example code, we perform two training rounds with slightly different models, allowing us to compare the results in the UI.   
"""

mlflow.autolog() # https://mlflow.org/docs/latest/tracking/autolog.html#automatic-logging
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# Let's take two slightly different models to compare. You can either use the example code or insert your own.

(X_train, y_train), (X_test, y_test) = mnist.load_data()

X_train = X_train / 255.
X_test = X_test / 255.
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

model_1 = Sequential(
    [
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation='relu'),
        layers.Dense(10, activation='softmax')
    ]
)
model_1.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

model_2 = Sequential(
    [
        layers.Flatten(input_shape=(28, 28)),
        layers.Dense(128, activation='tanh'),
        layers.Dense(64, activation='tanh'),
        layers.Dense(10, activation='softmax')
    ]
)
model_2.compile(optimizer='adam', 
                loss='categorical_crossentropy', 
                metrics=['accuracy'])

models = {model_1: "sequential_3layers", 
          model_2: "sequential_with_tanh"}

for model, name in models.items():
        
    with mlflow.start_run(): # Trigger MLflow to start tracking the run: https://mlflow.org/docs/latest/python_api/mlflow.html?highlight=autolog#mlflow.start_run
        
        run_name = name # Assign an informative name to the run; otherwise, a random name will be generated.
        mlflow.set_tag("mlflow.runName", run_name) 
        print(f"Run name: {run_name}")

        model.fit(X_train, y_train, epochs=5, batch_size=1, validation_data=(X_test, y_test))

        test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
        print('\nTest accuracy:', test_acc)
        
        mlflow.end_run() 

Run name: sequential_3layers
Epoch 1/5
    1/60000 [..............................] - ETA: 7:13:00 - loss: 2.0011 - accuracy: 0.0000e+00WARNING:tensorflow:Callback method `on_train_batch_end` is slow compared to the batch time (batch time: 0.0016s vs `on_train_batch_end` time: 0.0018s). Check your callbacks.


60000/60000 [==============================] - 124s 2ms/step - loss: 0.2099 - accuracy: 0.9392 - val_loss: 0.1408 - val_accuracy: 0.9592
Epoch 2/5
   83/60000 [..............................] - ETA: 1:52 - loss: 0.1127 - accuracy: 0.9639

/usr/local/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


60000/60000 [==============================] - 126s 2ms/step - loss: 0.1259 - accuracy: 0.9679 - val_loss: 0.1318 - val_accuracy: 0.9695
Epoch 3/5
60000/60000 [==============================] - 126s 2ms/step - loss: 0.1058 - accuracy: 0.9746 - val_loss: 0.1657 - val_accuracy: 0.9668
Epoch 4/5
60000/60000 [==============================] - 126s 2ms/step - loss: 0.0934 - accuracy: 0.9780 - val_loss: 0.1629 - val_accuracy: 0.9706
Epoch 5/5
1/1 [==============================] - 0s 46ms/step
INFO:tensorflow:Assets written to: /tmp/esternad/22951873/tmpp478ag7k/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/esternad/22951873/tmpp478ag7k/model/data/model/assets


...

60000/60000 [==============================] - 133s 2ms/step - loss: 0.2455 - accuracy: 0.9253 - val_loss: 0.1912 - val_accuracy: 0.9421
Epoch 2/5
   77/60000 [..............................] - ETA: 2:00 - loss: 0.0748 - accuracy: 0.9870

/usr/local/lib/python3.9/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


60000/60000 [==============================] - 131s 2ms/step - loss: 0.1488 - accuracy: 0.9564 - val_loss: 0.1630 - val_accuracy: 0.9509
Epoch 3/5
60000/60000 [==============================] - 134s 2ms/step - loss: 0.1327 - accuracy: 0.9601 - val_loss: 0.1565 - val_accuracy: 0.9553
Epoch 4/5
60000/60000 [==============================] - 135s 2ms/step - loss: 0.1187 - accuracy: 0.9639 - val_loss: 0.1192 - val_accuracy: 0.9656
Epoch 5/5
1/1 [==============================] - 0s 52ms/step
INFO:tensorflow:Assets written to: /tmp/esternad/22951873/tmpe097gz96/model/data/model/assets


INFO:tensorflow:Assets written to: /tmp/esternad/22951873/tmpe097gz96/model/data/model/assets


313/313 - 1s - loss: 0.1335 - accuracy: 0.9638 - 531ms/epoch - 2ms/step

Test accuracy: 0.9638000130653381


### User Interface

Next, we'll launch the MLflow application on Puhti and explore its user interface. You can find the MLflow icon under the Apps menu in Puhti. Ensure that the `pytorch/2.4` module is selected in the Settings section, as it includes a sufficiently recent version of MLflow. Additionally, make sure the file path points to the previously specified `mlruns` directory.

![mlflow_puhti](./pics/mlflow_puhti.png)

Picture 1. Setup MLflow in Puhti

After launching successfully, we can view the previous runs:

![run_1.png](./pics/runs_1.png)

Picture 2. Front page of MLflow UI

**Image caption:**

1. All experiments are listed here. With informative names and optional descriptions, user can organize different runs into easily manageable collections.
2. When selecting an experiment, all associated runs are displayed. User can sort and group these runs in various ways.

In the Charts view (see picture below), users can compare the performance of different models using automatically generated graphs. These graphs can be downloaded in formats like CSV. 

![run_2.png](./pics/run_2.png)

Picture 3. Charts- view of runs

#### Conclusion

The Tracking Server and Models components together provide an easy way to centrally monitor and store machine learning experiments in a consistent manner. These tools are user-friendly and don’t require deep expertise to get started. However, MLflow also offers the flexibility for more detailed configuration to meet the needs of more demanding use cases.

---


## Model Registry